In [56]:
from python_graphql_client import GraphqlClient
import datetime
import pandas
import IPython

# Instantiate the client with an endpoint.
clientv2 = GraphqlClient(endpoint="https://api.entur.io/journey-planner/v2/graphql")
#clientv3 = GraphqlClient(endpoint="https://api.entur.io/journey-planner/v3/graphql")

timestamp = "2021-08-30T15:30:00+02:00"
#timestamp = datetime.datetime.now().astimezone().isoformat()

# Create the query string and variables required for the request.
query = """
query ($id: String!, $timestamp: DateTime) {
  quay(id: $id) {
    estimatedCalls(numberOfDepartures: 20, startTime: $timestamp) {
      expectedArrivalTime
      aimedArrivalTime
      destinationDisplay {
        frontText
      }
      serviceJourney {
        line {
          publicCode
        }
      }
    }
  }
}


"""

# 103009 = Buran
variables = {
    "id": "NSR:Quay:103009",
    "timestamp": timestamp,
}

datav2 = clientv2.execute(query=query, variables=variables)
# datav3 = clientv3.execute(query=query, variables=variables)

datav2





{'data': {'quay': {'estimatedCalls': [{'expectedArrivalTime': '2021-08-30T15:29:45+0200',
     'aimedArrivalTime': '2021-08-30T15:30:00+0200',
     'destinationDisplay': {'frontText': 'Tillerterminalen via sentrum'},
     'serviceJourney': {'line': {'publicCode': '1'}}},
    {'expectedArrivalTime': '2021-08-30T15:31:56+0200',
     'aimedArrivalTime': '2021-08-30T15:32:00+0200',
     'destinationDisplay': {'frontText': 'Vestlia via sentrum-Othilienborg'},
     'serviceJourney': {'line': {'publicCode': '22'}}},
    {'expectedArrivalTime': '2021-08-30T15:34:06+0200',
     'aimedArrivalTime': '2021-08-30T15:33:00+0200',
     'destinationDisplay': {'frontText': 'Marienborg via sentrum'},
     'serviceJourney': {'line': {'publicCode': '12'}}},
    {'expectedArrivalTime': '2021-08-30T15:35:30+0200',
     'aimedArrivalTime': '2021-08-30T15:31:00+0200',
     'destinationDisplay': {'frontText': 'Lund via Kolstad-Heimdal'},
     'serviceJourney': {'line': {'publicCode': '2'}}},
    {'expectedArri

In [57]:
calls = datav2['data']['quay']['estimatedCalls']
output = []

for arrival in calls:
    expectedTime = arrival['expectedArrivalTime'].split('T')[1].split('+')[0]
    lineNr = arrival['serviceJourney']['line']['publicCode']
    lineName = arrival['destinationDisplay']['frontText']
    output.append([lineNr, lineName, expectedTime])
    
pandas.DataFrame(output, columns=["LinjeNr", "Navn", "Estimert tid"])

# sammenlikn med
# https://rp.atb.no/scripts/TravelMagic/TravelMagicWE.dll/svar?referrer=&lang=no&dep1=1&theme=&from=Buran+1+%28Trondheim%29&to=&Time=14%3A45&Date=30.08.2021&direction=1&search=Vis+reiseforslag&GetTR0=1&GetTR1=1&GetTR2=1&GetTR3=1&GetTR4=1&GetTR5=1&GetTR6=1&GetTR7=1&through=&throughpause=&changepenalty=1&changepause=0&linjer=&destinations=


    

,LinjeNr,Navn,Estimert tid
0,1,Tillerterminalen via sentrum,15:29:45
1,22,Vestlia via sentrum-Othilienborg,15:31:56
2,12,Marienborg via sentrum,15:34:06
3,2,Lund via Kolstad-Heimdal,15:35:30
4,10,Ratesvingen via sentrum,15:35:56
5,1,Kattem via sentrum-Tiller,15:36:12
6,70,Marienborg via sentrum,15:36:12
7,2,Midteggen via sentrum,15:36:09
8,20,Romolslia via Tempevegen,15:39:19
9,1,Tillerterminalen via sentrum,15:39:45


In [58]:
#url = 'https://rp.atb.no/scripts/TravelMagic/TravelMagicWE.dll/svar?referrer=&lang=no&dep1=1&theme=&from=Buran+1+%28Trondheim%29&to=&Time=14%3A45&Date=30.08.2021&direction=1&search=Vis+reiseforslag&GetTR0=1&GetTR1=1&GetTR2=1&GetTR3=1&GetTR4=1&GetTR5=1&GetTR6=1&GetTR7=1&through=&throughpause=&changepenalty=1&changepause=0&linjer=&destinations='

dt = datetime.datetime.fromisoformat(timestamp)
time = str(dt.hour) + "%3A" + str(dt.minute)
date = str(dt.day) + "." + str(dt.month) + "." + str(dt.year)

url = "https://rp.atb.no/scripts/TravelMagic/TravelMagicWE.dll/svar?referrer=&lang=no&dep1=1&theme=&from=Buran+1+%28Trondheim%29&to="
url = url + "&Time=" + time
url = url + "&Date=" + date
url = url + "&direction=1&search=Vis+reiseforslag&GetTR0=1&GetTR1=1&GetTR2=1&GetTR3=1&GetTR4=1&GetTR5=1&GetTR6=1&GetTR7=1&through=&throughpause=&changepenalty=1&changepause=0&linjer=&destinations="

IPython.display.IFrame(url, 700, 350)